##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Lite Model Maker를 사용한 BERT 질문 답변

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/lite/tutorials/model_maker_question_answer"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/tutorials/model_maker_question_answer.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/tutorials/model_maker_question_answer.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorflow/tensorflow/lite/g3doc/tutorials/model_maker_question_answer.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

The [TensorFlow Lite Model Maker library](https://www.tensorflow.org/lite/guide/model_maker) simplifies the process of adapting and converting a TensorFlow model to particular input data when deploying this model for on-device ML applications.

이 노트북에서는 질문 답변 작업에 일반적으로 사용되는 질문 답변 모델을 조정하고 변환하는 데 Model Maker 라이브러리를 사용하는 방법을 보여주는 엔드 투 엔드 예를 소개합니다.

# BERT 질문 답변 작업 소개

이 라이브러리에서 지원되는 작업은 질문에 대한 답변을 추출하는 작업입니다. 즉, 구절과 질문이 주어졌을 때 구절의 범위가 답변으로 제공됩니다. 아래 이미지는 질문 답변의 예입니다.

<p align="center"><img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_squad_showcase.png" width="500"></p>

<p align="center">
    <em>답변은 구절의 범위입니다(이미지 제공: <a href="https://rajpurkar.github.io/mlx/qa-and-squad/">SQuAD 블로그</a>).</em>
</p>

질문 답변 작업 모델의 경우, 입력은 이미 전처리된 구절과 질문 쌍이어야 하며 출력은 구절의 각 토큰에 대한 시작 로짓과 끝 로짓이어야 합니다. 구절과 질문의 길이에 따라 입력 크기를 설정하고 조정할 수 있습니다.

## 엔드 투 엔드 개요


다음 코드 조각은 몇 줄의 코드로 모델을 가져오는 방법을 보여줍니다. 전체 프로세스에는 (1) 모델 선택, (2) 데이터 로드, (3) 모델 재훈련, (4) 평가, (5) TensorFlow Lite 형식으로 내보내기의 5 단계가 포함됩니다.

```python
# Chooses a model specification that represents the model.
spec = model_spec.get('mobilebert_qa')

# Gets the training data and validation data.
train_data = DataLoader.from_squad(train_data_path, spec, is_training=True)
validation_data = DataLoader.from_squad(validation_data_path, spec, is_training=False)

# Fine-tunes the model.
model = question_answer.create(train_data, model_spec=spec)

# Gets the evaluation result.
metric = model.evaluate(validation_data)

# Exports the model to the TensorFlow Lite format with metadata in the export directory.
model.export(export_dir)
```

다음 섹션에서 코드를 자세히 설명합니다.

## 전제 조건

이 예제를 실행하려면 [GitHub 리포지토리](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker)의 Model Maker 패키지를 포함하여 필요한 패키지를 설치해야 합니다.

In [ ]:
!pip install -q tflite-model-maker

필요한 패키지를 가져옵니다.

In [ ]:
import numpy as np
import os

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import question_answer
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.question_answer import DataLoader

"엔드 투 엔드 개요"는 간단한 엔드 투 엔드 예제를 보여줍니다. 다음 섹션에서는 예제를 단계별로 진행하면서 자세히 설명합니다.

## 질문 답변 모델을 나타내는 model_spec 선택하기

각 `model_spec` 객체는 질문 답변에 대한 특정 모델을 나타냅니다. Model Maker는 현재 MobileBERT 및 BERT-Base 모델을 지원합니다.

지원 모델 | model_spec의 이름 | 모델 설명
--- | --- | ---
[MobileBERT](https://arxiv.org/pdf/2004.02984.pdf) | 'mobilebert_qa' | BERT-Base보다 4.3배 더 작고 5.5배 더 빠르면서도 기기 내 시나리오에 적합한 경쟁력 있는 결과를 제공합니다.
[MobileBERT-SQuAD](https://arxiv.org/pdf/2004.02984.pdf) | 'mobilebert_qa_squad' | MobileBERT 모델과 동일한 모델 아키텍처이며 초기 모델은 이미 [SQuAD1.1](https://rajpurkar.github.io/SQuAD-explorer/)에서 재훈련되었습니다.
[BERT-Base](https://arxiv.org/pdf/1810.04805.pdf) | 'bert_qa' | NLP 작업에서 널리 사용되는 표준 BERT 모델입니다.

이 튜토리얼에서는 [MobileBERT-SQuAD](https://arxiv.org/pdf/2004.02984.pdf)가 예로 사용됩니다. 모델이 이미 [SQuAD1.1](https://rajpurkar.github.io/SQuAD-explorer/)에서 재훈련되었으므로 질문 답변 작업을 더 빨리 처리할 수 있습니다.


In [ ]:
spec = model_spec.get('mobilebert_qa_squad')

## 기기 내 ML 앱에 특정한 입력 데이터 로드 및 데이터 전처리하기

[TriviaQA](https://nlp.cs.washington.edu/triviaqa/)는 650K 이상의 질문-답변-증거 삼중 요소를 포함한 독해 데이터세트입니다. 이 튜토리얼에서는 이 데이터세트의 일부를 사용하여 Model Maker 라이브러리의 사용 방법을 학습합니다.

데이터를 로드하려면 `--sample_size=8000` 및 `web` 데이터세트로 [변환기 Python 스크립트](https://github.com/mandarjoshi90/triviaqa#miscellaneous)를 실행하여 TriviaQA 데이터세트를 [SQuAD1.1](https://rajpurkar.github.io/SQuAD-explorer/)로 변환합니다. 다음과 같이 변환 코드를 약간 수정합니다.

- 컨텍스트 문서에서 답변을 찾을 수 없는 샘플은 건너뜁니다.
- 대문자나 소문자 없이 컨텍스트에서 원래 답변을 얻습니다.

이미 변환된 데이터세트의 보관된 버전을 다운로드합니다.

In [ ]:
train_data_path = tf.keras.utils.get_file(
    fname='triviaqa-web-train-8000.json',
    origin='https://storage.googleapis.com/download.tensorflow.org/models/tflite/dataset/triviaqa-web-train-8000.json')
validation_data_path = tf.keras.utils.get_file(
    fname='triviaqa-verified-web-dev.json',
    origin='https://storage.googleapis.com/download.tensorflow.org/models/tflite/dataset/triviaqa-verified-web-dev.json')

고유한 데이터세트로 MobileBERT 모델을 훈련할 수도 있습니다. Colab에서 이 노트북을 실행하는 경우, 왼쪽 사이드바를 사용하여 데이터를 업로드합니다.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_question_answer.png" alt="파일 업로드" width="800" hspace="100">

클라우드에 데이터를 업로드하지 않으려면 [가이드](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker)에 따라 오프라인으로 라이브러리를 실행할 수도 있습니다.

`DataLoader.from_squad` 메서드를 사용하여 특정 `model_spec`에 따라 [SQuAD 형식](https://rajpurkar.github.io/SQuAD-explorer/) 데이터를 로드하고 전처리합니다. SQuAD2.0 또는 SQuAD1.1 형식을 사용할 수 있습니다. 매개변수 `version_2_with_negative`를 `True`로 설정하면 형식이 SQuAD2.0임을 의미합니다. 그렇지 않으면 형식은 SQuAD1.1입니다. 기본적으로, `version_2_with_negative`는 `False`입니다.

In [ ]:
train_data = DataLoader.from_squad(train_data_path, spec, is_training=True)
validation_data = DataLoader.from_squad(validation_data_path, spec, is_training=False)

## TensorFlow 모델 사용자 정의하기

로드된 데이터를 기반으로 사용자 정의 질문 답변 모델을 만듭니다. `create` 함수는 다음 단계로 구성됩니다.

1. Creates the model for question answer according to `model_spec`.
2. 질문 답변 모델을 훈련합니다. 기본 epoch 및 기본 배치 크기는 `model_spec` 객체의 두 변수 `default_training_epochs` 및 `default_batch_size`에 따라 설정됩니다.

In [ ]:
model = question_answer.create(train_data, model_spec=spec)

자세한 모델 구조를 살펴보겠습니다.

In [ ]:
model.summary()

## 사용자 정의 모델 평가하기

검증 데이터에서 모델을 평가하고 `f1` 점수 및 `exact match` 등을 포함한 메트릭 사전을 가져옵니다. 메트릭은 SQuAD1.1 및 SQuAD2.0 간에 서로 다릅니다.

In [ ]:
model.evaluate(validation_data)

## TensorFlow Lite 모델로 내보내기

나중에 기기 내 ML 애플리케이션에서 사용할 수 있도록 훈련된 모델을 [메타데이터](https://www.tensorflow.org/lite/convert/metadata)가 포함된 TensorFlow Lite 모델 형식으로 변환합니다. 어휘 파일은 메타데이터에 포함됩니다. 기본 TFLite 파일 이름은 `model.tflite`입니다.

많은 기기 내 ML 애플리케이션에서 모델 크기는 중요한 요소입니다. 따라서 모델을 더 작게 만들고 잠재적으로 더 빠르게 실행하려면 양자화를 적용하는 것이 좋습니다. 기본 훈련 후 양자화 기술은 BERT 및 MobileBERT 모델에 대한 동적 범위 양자화입니다.

In [ ]:
model.export(export_dir='.')

[TensorFlow Lite 작업 라이브러리](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview)의 [BertQuestionAnswerer API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_question_answerer)를 사용하여 [bert_qa](https://github.com/tensorflow/examples/tree/master/lite/examples/bert_qa/android) 참조 앱에서 TensorFlow Lite 모델 파일을 사용할 수 있습니다. Colab 왼쪽 사이드바에서 이를 다운로드할 수 있습니다.

The allowed export formats can be one or a list of the following:

- `ExportFormat.TFLITE`
- `ExportFormat.VOCAB`
- `ExportFormat.SAVED_MODEL`

기본적으로, 메타데이터와 함께 TensorFlow Lite 모델만 내보냅니다. 다른 파일을 선택적으로 내보낼 수도 있습니다. 예를 들어, 다음과 같이 vocab 파일만 내보냅니다.

In [ ]:
model.export(export_dir='.', export_format=ExportFormat.VOCAB)

You can also evaluate the tflite model with the `evaluate_tflite` method. This step is expected to take a long time.

In [ ]:
model.evaluate_tflite('model.tflite', validation_data)

## 고급 사용

`create` 함수는 이 라이브러리의 중요한 부분으로, `model_spec` 매개변수가 모델 사양을 정의합니다. `BertQASpec` 클래스는 현재 지원됩니다. MobileBERT 모델과 BERT-Base 모델의 두 가지 모델이 있습니다. `create` 함수는 다음 단계로 구성됩니다.

1. `model_spec`에 따라 질문 답변 모델을 생성합니다.
2. 질문 답변 모델을 훈련합니다.

이 섹션에서는 모델 조정, 훈련 하이퍼 매개변수 조정 등의 몇 가지 고급 주제를 설명합니다.

### 모델 조정하기

`BertQASpec` 클래스에서 매개변수 `seq_len` 및 `query_len`과 같은 모델 인프라를 조정할 수 있습니다.

모델의 조정 가능한 매개변수:

- `seq_len`: 모델에 공급할 구절의 길이입니다.
- `query_len`: 모델에 공급할 질문의 길이입니다.
- `doc_stride`: 슬라이딩 윈도우 방식으로 문서 청크를 가져올 때의 폭입니다.
- `initializer_range`: 모든 가중치 행렬을 초기화하기 위한 truncated_normal_initializer의 표준 편차입니다.
- `trainable`: 사전 훈련된 레이어가 훈련 가능한지 여부를 지정하는 부울 값입니다.

훈련 파이프라인의 조정 가능한 매개변수:

- `model_dir`: 모델 체크포인트 파일의 위치입니다. 설정하지 않으면 임시 디렉토리가 사용됩니다.
- `dropout_rate`: 드롭아웃 비율입니다.
- `learning_rate`: Adam의 초기 학습률입니다.
- `predict_batch_size`: 예측을 위한 배치 크기입니다.
- `tpu`: 연결할 TPU 주소입니다. tpu를 사용하는 경우에만 사용됩니다.


예를 들어, 더 긴 시퀀스 길이로 모델을 훈련할 수 있습니다. 모델을 변경하는 경우, 먼저 새 `model_spec`을 구성해야 합니다.

In [ ]:
new_spec = model_spec.get('mobilebert_qa')
new_spec.seq_len = 512

나머지 단계는 같습니다. 모델 사양에 따라 전처리 단계가 다를 수 있으므로 `dataloader` 및 `create` 부분 모두 다시 실행해야 합니다.


### 훈련 하이퍼 매개변수 조정하기

모델 성능에 영향을 미치는 `epochs` 및 `batch_size`와 같은 훈련 하이퍼 매개변수를 조정할 수도 있습니다. 예를 들면 다음과 같습니다.

- `epochs`: epoch가 많을수록 정확성이 향상될 수 있지만, 과대적합으로 이어질 수 있습니다.
- `batch_size`: 하나의 훈련 단계에서 사용할 샘플의 수입니다.

예를 들어, 다음과 같이 더 많은 epoch와 더 큰 배치 크기로 훈련할 수 있습니다.

```python
model = question_answer.create(train_data, model_spec=spec, epochs=5, batch_size=64)
```

### 모델 아키텍처 변경하기

`model_spec`을 변경하여 데이터 훈련의 기본 모델을 변경할 수 있습니다. 예를 들어, BERT-Base 모델로 변경하려면 다음을 실행합니다.

```python
spec = model_spec.get('bert_qa')
```

나머지 단계는 같습니다.

### TensorFlow Lite 모델에서 훈련 후 양자화 사용자 정의하기

[훈련 후 양자화](https://www.tensorflow.org/lite/performance/post_training_quantization)는 모델 정확도를 약간만 떨어트리면서 모델 크기와 추론 지연 시간을 줄이는 동시에 CPU 및 하드웨어 가속기의 추론 속도도 개선할 수 있는 변환 기술입니다. 따라서 모델을 최적화하는 데 널리 사용됩니다.

Model Maker 라이브러리는 모델을 내보낼 때 기본 훈련 후 양자화 기술을 적용합니다. 훈련 후 양자화를 사용자 지정하려는 경우 Model Maker는 [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig)를 사용하여 여러 훈련 후 양자화 옵션도 지원합니다. float16 양자화를 예로 들어보겠습니다. 먼저 양자화 구성을 정의합니다.

```python
config = QuantizationConfig.for_float16()
```

그런 다음 이러한 구성을 가진 TensorFlow Lite 모델을 내보냅니다.

```python
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)
```

# 더 읽어보기

기술적인 세부 사항을 배우려면 [BERT 질문 및 답변](https://www.tensorflow.org/lite/examples/bert_qa/overview) 예제를 읽을 수 있습니다. 자세한 내용은 다음을 참조하세요.

- TensorFlow Lite Model Maker [가이드](https://www.tensorflow.org/lite/guide/model_maker) 및 [API 참조](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker)
- 작업 라이브러리: 배포용 [BertQuestionAnswerer](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_question_answerer)
- 엔드 투 엔드 참조 앱: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/bert_qa/android) 및 [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/bert_qa/ios)